# Fonctions d'acquisition pour l'apprentissage actif sur des images de déchets

Ce notebook présente et implémente différentes fonctions d'acquisition utilisées dans le cadre de l'apprentissage actif pour la classification d'images de déchets. Les fonctions d'acquisition permettent de sélectionner les échantillons les plus informatifs à annoter, afin d'améliorer l'efficacité de l'entraînement du modèle tout en réduisant le nombre d'annotations nécessaires.

## 1. Définir les fonctions d'acquisition

Les fonctions d'acquisition jouent un rôle central dans l'apprentissage actif. Elles permettent de mesurer l'incertitude du modèle sur les données non annotées et de sélectionner les exemples les plus pertinents à ajouter au jeu d'entraînement.

Dans ce notebook, nous allons implémenter les fonctions d'acquisition suivantes :
- **Max Entropy** : sélectionne les échantillons ayant la plus grande entropie prédictive.
- **BALD** (Bayesian Active Learning by Disagreement) : sélectionne les échantillons maximisant l'information mutuelle entre les prédictions et les paramètres du modèle.
- **Mean STD** : sélectionne les échantillons ayant la plus grande variance moyenne des prédictions.
- **Variational Ratios** : sélectionne les échantillons pour lesquels le modèle a le moins de confiance (plus grande incertitude).

## 2. Implémenter la fonction de prédiction sur le pool

Pour toutes les fonctions d'acquisition, il est nécessaire d'obtenir des prédictions du modèle sur un sous-ensemble aléatoire du pool d'images. On utilise le MC Dropout pour obtenir une distribution de sorties, ce qui permet d'estimer l'incertitude du modèle.

La fonction suivante effectue plusieurs passes stochastiques sur le modèle pour chaque échantillon du pool et retourne les probabilités prédites.

In [ ]:
import numpy as np
import torch

def predictions_from_pool(
    model, 
    X_pool: np.ndarray, 
    T: int = 100, 
    training: bool = True
):
    """
    Effectue des prédictions sur un sous-ensemble aléatoire du pool à l'aide du modèle,
    en utilisant MC Dropout pour obtenir des distributions de sortie.

    Args:
        model: Modèle entraîné (doit supporter le mode dropout).
        X_pool: Ensemble de données (pool) sous forme de tableau numpy.
        T: Nombre d'itérations MC Dropout.
        training: Si True, active le mode entraînement (dropout activé).

    Returns:
        outputs: Tableau numpy de forme (T, N_subset, C) contenant les probabilités.
        random_subset: Indices sélectionnés dans le pool.
    """
    subset_size = min(500, len(X_pool))
    random_subset = np.random.choice(range(len(X_pool)), size=subset_size, replace=False)
    with torch.no_grad():
        outputs = np.stack(
            [
                torch.softmax(
                    model.estimator.forward(X_pool[random_subset], training=training),
                    dim=-1,
                )
                .cpu()
                .numpy()
                for _ in range(T)
            ]
        )
    return outputs, random_subset

## 3. Fonction d'entropie de Shannon

L'entropie de Shannon mesure l'incertitude globale des prédictions du modèle pour chaque échantillon. Plus l'entropie est élevée, plus le modèle est incertain.

La fonction suivante calcule l'entropie de Shannon à partir des prédictions obtenues via MC Dropout.

In [ ]:
def shannon_entropy_function(
    model, 
    X_pool: np.ndarray, 
    T: int = 100, 
    E_H: bool = False, 
    training: bool = True
):
    """
    Calcule l'entropie de Shannon sur les prédictions du modèle pour mesurer l'incertitude.

    Args:
        model: Modèle entraîné.
        X_pool: Pool d'échantillons.
        T: Nombre d'itérations MC Dropout.
        E_H: Si True, retourne aussi l'entropie attendue pour BALD.
        training: Mode entraînement (dropout activé).

    Returns:
        H: Entropie de Shannon pour chaque échantillon.
        E (optionnel): Entropie attendue (pour BALD).
        random_subset: Indices sélectionnés dans le pool.
    """
    outputs, random_subset = predictions_from_pool(model, X_pool, T, training=training)
    pc = outputs.mean(axis=0)
    H = (-pc * np.log(pc + 1e-10)).sum(axis=-1) # Pour éviter log(0)
    if E_H:
        E = -np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0)
        return H, E, random_subset
    return H, random_subset

## 4. Fonction Max Entropy

La stratégie Max Entropy sélectionne les échantillons du pool pour lesquels l'entropie prédictive est maximale, c'est-à-dire ceux pour lesquels le modèle est le plus incertain.

Voici l'implémentation de cette fonction d'acquisition :

In [ ]:
def max_entropy(
    model, X_pool: np.ndarray, n_query: int = 10, T: int = 100, training: bool = True
):
    """
    Sélectionne les points du pool qui maximisent l'entropie prédictive.

    Args:
        model: Modèle entraîné.
        X_pool: Pool d'échantillons.
        n_query: Nombre d'échantillons à sélectionner.
        T: Nombre d'itérations MC Dropout.
        training: Mode entraînement (dropout activé).

    Returns:
        query_idx: Indices des échantillons sélectionnés dans le pool.
        X_pool[query_idx]: Données correspondantes.
    """
    acquisition, random_subset = shannon_entropy_function(
        model, X_pool, T, training=training
    )
    idx = (-acquisition).argsort()[:n_query]
    query_idx = random_subset[idx]
    return query_idx, X_pool[query_idx]

## 5. Fonction Mean STD Acquisition

La stratégie Mean STD sélectionne les échantillons pour lesquels la variance moyenne des prédictions (sur les classes) est la plus élevée, ce qui indique une forte incertitude du modèle.

Voici l'implémentation de cette fonction :

In [ ]:
def mean_std_acquisition(
    model, 
    X_pool: np.ndarray, 
    n_query: int = 10, 
    T: int = 100, 
    training: bool = True
):
    """
    Sélectionne les points du pool qui maximisent l'écart-type moyen des prédictions (Mean STD).

    Args:
        model: Modèle entraîné.
        X_pool: Pool d'échantillons.
        n_query: Nombre d'échantillons à sélectionner.
        T: Nombre d'itérations MC Dropout.
        training: Mode entraînement (dropout activé).

    Returns:
        query_idx: Indices des échantillons sélectionnés dans le pool.
        X_pool[query_idx]: Données correspondantes.
    """
    outputs, random_subset = predictions_from_pool(model, X_pool, T, training=training)
    expected_p_c = np.mean(outputs, axis=0)
    expected_p_c_squared = np.mean(outputs**2, axis=0)
    sigma_c = expected_p_c_squared - (expected_p_c**2)
    acquisition_scores = np.mean(sigma_c, axis=-1)
    idx = (-acquisition_scores).argsort()[:n_query]
    query_idx = random_subset[idx]
    return query_idx, X_pool[query_idx]

## 6. Fonction BALD

La stratégie BALD (Bayesian Active Learning by Disagreement) sélectionne les échantillons qui maximisent l'information mutuelle entre les prédictions et les paramètres du modèle. Cela permet de cibler les exemples pour lesquels le modèle est incertain en moyenne, mais où il existe une forte désaccord entre différentes passes du modèle.

Voici l'implémentation de la fonction BALD :

In [ ]:
def bald(
    model, X_pool: np.ndarray, n_query: int = 10, T: int = 100, training: bool = True
):
    """
    Sélectionne les points du pool qui maximisent l'information mutuelle (BALD).

    Args:
        model: Modèle entraîné.
        X_pool: Pool d'échantillons.
        n_query: Nombre d'échantillons à sélectionner.
        T: Nombre d'itérations MC Dropout.
        training: Mode entraînement (dropout activé).

    Returns:
        query_idx: Indices des échantillons sélectionnés dans le pool.
        X_pool[query_idx]: Données correspondantes.
    """
    H, E_H, random_subset = shannon_entropy_function(
        model, X_pool, T, E_H=True, training=training
    )
    acquisition = H - E_H
    idx = (-acquisition).argsort()[:n_query]
    query_idx = random_subset[idx]
    return query_idx, X_pool[query_idx]

## 7. Fonction Variational Ratios

La stratégie Variational Ratios mesure le manque de confiance du modèle en sélectionnant les échantillons pour lesquels la classe prédite la plus fréquente a la plus faible proportion parmi les passes MC Dropout.

Voici l'implémentation de cette fonction :

In [ ]:
from scipy import stats

def var_ratios(
    model, X_pool: np.ndarray, n_query: int = 10, T: int = 100, training: bool = True
):
    """
    Sélectionne les points du pool pour lesquels le modèle a le moins de confiance (Variational Ratios).

    Args:
        model: Modèle entraîné.
        X_pool: Pool d'échantillons.
        n_query: Nombre d'échantillons à sélectionner.
        T: Nombre d'itérations MC Dropout.
        training: Mode entraînement (dropout activé).

    Returns:
        query_idx: Indices des échantillons sélectionnés dans le pool.
        X_pool[query_idx]: Données correspondantes.
    """
    outputs, random_subset = predictions_from_pool(model, X_pool, T, training)
    preds = np.argmax(outputs, axis=2)
    _, count = stats.mode(preds, axis=0)
    acquisition = (1 - count / preds.shape[1]).reshape((-1,))
    idx = (-acquisition).argsort()[:n_query]
    query_idx = random_subset[idx]
    return query_idx, X_pool[query_idx]